In [69]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm_notebook as tqdm

In [70]:
path = '.\\done_oct\\'

In [71]:
path2 = '.\\data\\oct\\16\\'

In [4]:
pwd

'C:\\Users\\user\\Documents\\GIT\\city\\user_scripts\\masha_k\\Transnavigation\\algoritm'

## ИЗМЕНИТЬ НАЗВАНИЯ ФАЙЛА!!! НЕ ЗАБЫТЬ

Считываем данные по трекам с привязанными остановками

In [5]:
res_stop = pd.read_csv(path+'res_2019_10_16.csv', sep = ';', encoding='utf-8')

In [6]:
res_stop.head()

,mr_id,rc_orderby,ssId,tr_id,timenav,len,geometry,ss_lat,ss_long,tabelnum,rl_id
0,100,1,1460,500000338,2019-10-16 06:06:22,13,"LINESTRING (50.155472 53.13672, 50.155472 53.1...",53.136825,50.155377,10193.0,59565
1,100,1,1460,500000338,2019-10-16 06:06:24,11,"LINESTRING (50.155488 53.13675, 50.155488 53.1...",53.136825,50.155377,10193.0,59565
2,100,1,1460,500000338,2019-10-16 06:06:26,9,"LINESTRING (50.155498 53.136772, 50.155498 53....",53.136825,50.155377,10193.0,59565
3,100,1,1460,500000338,2019-10-16 06:06:27,7,"LINESTRING (50.155492 53.13681800000001, 50.15...",53.136825,50.155377,10193.0,59565
4,100,1,1460,500000338,2019-10-16 06:06:33,4,"LINESTRING (50.155435 53.136842, 50.155435 53....",53.136825,50.155377,10193.0,59565


Определяем времена начала и конца для каждого табельного номера

In [7]:
Samara_mr_id_marsh_valid_id = pd.read_csv(path+'Samara_mr_id_marsh_valid_id.csv', sep = ';', encoding = 'utf-8')

In [8]:
Samara_mr_id_marsh_valid_id.head()

,code_marsh,mr_id
0,10003456789,1
1,112103456789,1
2,3803456789,10
3,113203456789,187
4,11103456789,187


In [9]:
len(Samara_mr_id_marsh_valid_id)

161

In [10]:
Samara_mr_id_marsh_valid_id['Код Маршрута']=Samara_mr_id_marsh_valid_id['code_marsh']

In [11]:
Samara_mr_id_marsh_valid_id=Samara_mr_id_marsh_valid_id.drop('code_marsh',axis=1)

## ИЗМЕНИИТЬ НАЗВАНИЯ ФАЙЛА!!! НЕ ЗАБЫТЬ

In [12]:
validation= pd.read_csv(path+'2019.10.16.csv', sep = ',', encoding = 'cp1251')
#validation=validation.merge(Samara_mr_id_marsh_valid_id,left_on='Код Маршрута',right_on ='code_marsh', how='left')
#validation=validation.drop('code_marsh',axis=1)
validation=validation.merge(Samara_mr_id_marsh_valid_id,on =['Код Маршрута'])
validation=validation[validation['mr_id'].isin(res_stop['mr_id'].unique())]
validation['time']=pd.to_datetime(validation['Дата и время поездки'])
validation['hour']=validation['time'].dt.hour
validation.loc[validation.hour ==0, 'hour'] = 24
validation.loc[validation.hour ==1, 'hour'] = 25
validation.loc[validation.hour ==2, 'hour'] = 26
validation.loc[validation.hour ==3, 'hour'] = 27
validation['min']=validation['time'].dt.minute
validation['sec']=validation['time'].dt.second
validation['time']=(validation['hour'].astype(int))*3600+(validation['min'].astype(int))*60+(validation['sec'].astype(int))
validation=validation.drop(['hour'], axis=1).drop(['min'], axis=1).drop(['sec'], axis=1)
validation_start=pd.DataFrame(validation.groupby(['Код Маршрута','Табельный номер']).agg({'time': np.min})).reset_index().rename(columns={"time": "start"})
validation_finish=pd.DataFrame(validation.groupby(['Код Маршрута','Табельный номер']).agg({'time': np.max})).reset_index().rename(columns={"time": "finish"})
validation_start_finish=validation_start.merge(validation_finish, on=['Код Маршрута','Табельный номер'])
validation2=validation.merge(validation_start_finish, on=['Код Маршрута','Табельный номер'])
del validation_start
del validation_finish
del validation

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [72]:
files = [f for f in os.listdir(path2) if "_4.csv" in f]
df_4=pd.DataFrame()
for file in files:
    m = pd.read_csv(path2+file, sep = '\t', encoding='windows-1251')
    df_4=df_4.append(m,ignore_index=True)
    del m
df_4['time']=pd.to_datetime(df_4['timenav'])
df_4['hour']=df_4['time'].dt.hour
df_4['min']=df_4['time'].dt.minute
df_4['sec']=df_4['time'].dt.second
df_4.loc[df_4.hour ==0, 'hour'] = 24
df_4.loc[df_4.hour ==1, 'hour'] = 25
df_4.loc[df_4.hour ==2, 'hour'] = 26
df_4.loc[df_4.hour ==3, 'hour'] = 27
df_4['time']=(df_4['hour'].astype(int))*3600+(df_4['min'].astype(int))*60+(df_4['sec'].astype(int))
#df_4=df_4.drop(['hour'], axis=1).drop(['min'], axis=1).drop(['sec'], axis=1)
df_4_start=pd.DataFrame(df_4.groupby(['mr_id','tabelnum']).agg({'time': np.min})).reset_index().rename(columns={"time": "start"})
df_4_finish=pd.DataFrame(df_4.groupby(['mr_id','tabelnum']).agg({'time': np.max})).reset_index().rename(columns={"time": "finish"})
df_4_start_finish=df_4_start.merge(df_4_finish, on=['mr_id','tabelnum']).sort_values(by=['mr_id', 'start'])
del df_4_start
del df_4_finish

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [75]:
df_4[df_4['mr_id']==19]

,tr_id,mr_id,mr_name,graph,tabelnum,condnum,timenav,long,lat,speed,mark,time,hour,min,sec
1687255,500000287,19,11,1,10074.0,0,2019-10-16 07:10:02,50.123267,53.194577,0,0,25802,7,10,2
1687256,500000287,19,11,1,10074.0,0,2019-10-16 07:10:11,50.123267,53.194577,0,0,25811,7,10,11
1687257,500000287,19,11,1,10074.0,0,2019-10-16 07:10:14,50.123267,53.194577,0,0,25814,7,10,14
1687258,500000287,19,11,1,10074.0,0,2019-10-16 07:10:22,50.132570,53.215573,26,0,25822,7,10,22
1687259,500000287,19,11,1,10074.0,0,2019-10-16 07:10:30,50.133842,53.216055,41,0,25830,7,10,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693216,500000287,19,11,1,10074.0,0,2019-10-16 19:49:11,50.123430,53.194643,0,0,71351,19,49,11
1693217,500000287,19,11,1,10074.0,0,2019-10-16 19:49:22,50.123430,53.194643,0,0,71362,19,49,22
1693218,500000287,19,11,1,10074.0,0,2019-10-16 19:49:32,50.123430,53.194643,0,0,71372,19,49,32
1693219,500000287,19,11,1,10074.0,0,2019-10-16 19:49:42,50.123430,53.194643,0,0,71382,19,49,42


In [14]:
validation_start_finish2=validation_start_finish.merge(Samara_mr_id_marsh_valid_id,on =['Код Маршрута'])
mrid=validation_start_finish2['mr_id'].unique()
df_4_start_finish=df_4_start_finish[df_4_start_finish['mr_id'].isin(mrid)]
validation_start_finish2=validation_start_finish2.sort_values(by=['mr_id', 'start'])
df_4_start_finish=df_4_start_finish.sort_values(by=['mr_id', 'start'])

In [15]:
validation_start_finish2.head()

,Код Маршрута,Табельный номер,start,finish,mr_id
341,10003456789,396,18901,61893,1
353,10003456789,500,18910,71673,1
344,10003456789,581,19902,72501,1
343,10003456789,553,20383,64603,1
357,10003456789,590,20546,74821,1


In [16]:
df_4_start_finish_mem=df_4_start_finish

In [17]:
validation_start_finish2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 341 to 303
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Код Маршрута     840 non-null    int64 
 1   Табельный номер  840 non-null    object
 2   start            840 non-null    int32 
 3   finish           840 non-null    int32 
 4   mr_id            840 non-null    int64 
dtypes: int32(2), int64(2), object(1)
memory usage: 32.8+ KB


In [18]:
df_4_start_finish_mem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 864 entries, 12 to 894
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mr_id     864 non-null    int64  
 1   tabelnum  864 non-null    float64
 2   start     864 non-null    int32  
 3   finish    864 non-null    int32  
dtypes: float64(1), int32(2), int64(1)
memory usage: 27.0 KB


In [19]:
df_4_start_finish.head()

,mr_id,tabelnum,start,finish
12,1,1408.0,18002,74220
13,1,1456.0,18303,65389
10,1,1329.0,18967,74995
16,1,1591.0,19145,67379
11,1,1357.0,19803,75954


In [20]:
res=validation_start_finish2

In [21]:
validation_start_finish2.head()

,Код Маршрута,Табельный номер,start,finish,mr_id
341,10003456789,396,18901,61893,1
353,10003456789,500,18910,71673,1
344,10003456789,581,19902,72501,1
343,10003456789,553,20383,64603,1
357,10003456789,590,20546,74821,1


1. Раскидываем машины по табельным номерам валидаций. Жесткое условие (все валидации внутри трека)

In [22]:
k=0
df_4_start_finish=df_4_start_finish_mem
res=validation_start_finish2
tabelnum = []
start = []
finish = []
tracks=pd.DataFrame()
for index, row in res.iterrows():
    stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['start']<=row['start'])
                    &(df_4_start_finish['finish']>row['finish'])]
    df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
    if stops.empty:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)
    else:
        tabelnum_unic=stops['tabelnum'].values
        tabelnum.append(tabelnum_unic[0])
        start_unic=stops['start'].values
        start.append(start_unic[0])
        finish_unic=stops['finish'].values
        finish.append(finish_unic[0])
res['tabelnum']=tabelnum
res['start_track']=start
res['finish_track']=finish

In [23]:
res.head()

,Код Маршрута,Табельный номер,start,finish,mr_id,tabelnum,start_track,finish_track
341,10003456789,396,18901,61893,1,1408.0,18002.0,74220.0
353,10003456789,500,18910,71673,1,0.0,0.0,0.0
344,10003456789,581,19902,72501,1,1329.0,18967.0,74995.0
343,10003456789,553,20383,64603,1,1456.0,18303.0,65389.0
357,10003456789,590,20546,74821,1,1357.0,19803.0,75954.0


In [24]:
len(df_4_start_finish)

201

2. Раскидываем машины по табельным номерам валидаций. Мягкое условие 1 (все валидации после начала трека)

In [25]:
k=0
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['start']<=row['start'])]
        df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_2']=tabelnum
res['start_track_2']=start
res['finish_track_2']=finish

3. Раскидываем машины по табельным номерам валидаций. Мягкое условие 2 (все валидации до конца трека)

In [26]:
k=0
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['finish']>row['finish'])]
        df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_3']=tabelnum
res['start_track_3']=start
res['finish_track_3']=finish

In [27]:
res.head()

,Код Маршрута,Табельный номер,start,finish,mr_id,tabelnum,start_track,finish_track,tabelnum_2,start_track_2,finish_track_2,tabelnum_3,start_track_3,finish_track_3
341,10003456789,396,18901,61893,1,1408.0,18002.0,74220.0,0.0,0.0,0.0,0.0,0.0,0.0
353,10003456789,500,18910,71673,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344,10003456789,581,19902,72501,1,1329.0,18967.0,74995.0,0.0,0.0,0.0,0.0,0.0,0.0
343,10003456789,553,20383,64603,1,1456.0,18303.0,65389.0,0.0,0.0,0.0,0.0,0.0,0.0
357,10003456789,590,20546,74821,1,1357.0,19803.0,75954.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
len(df_4_start_finish)

21

4.Дозаполняем табельные номера тем, что не вошло

In [29]:
k=0
df_4_start_finish=df_4_start_finish_mem
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['finish']>row['finish'])]
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_4']=tabelnum
res['start_track_4']=start
res['finish_track_4']=finish

In [30]:
res.to_csv(path+'for_mege_tabelnum_29.csv')
#df_4_start_finish.to_csv(path+'for_mege_tabelnum_ost_29.csv')

In [31]:
len(res[res['tabelnum_3']>0])

77

In [32]:
len(res[res['tabelnum']==0])

177

Перезаписываем данные в столбцах

In [33]:
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_3']
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_2']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_2']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_3']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_2']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_3']

In [34]:
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_4']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_4']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_4']

Пересохранили табельный номер как int, чтобы правильнее можно было merge делать

In [35]:
table_num_merge_29=res[['Табельный номер','mr_id','tabelnum']]

In [36]:
len(validation2['mr_id'].unique())

80

In [37]:
table_num_merge_29['Табельный номер']=table_num_merge_29['Табельный номер'].astype(int)

<ipython-input-37-f9f817fefa8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_num_merge_29['Табельный номер']=table_num_merge_29['Табельный номер'].astype(int)


In [38]:
table_num_merge_29[['Табельный номер','tabelnum']]

,Табельный номер,tabelnum
341,396,1408.0
353,500,1408.0
344,581,1329.0
343,553,1456.0
357,590,1357.0
...,...,...
304,161,10013.0
302,149,10353.0
305,602,10349.0
306,603,10203.0


In [39]:
table_num_merge_29.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 341 to 303
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Табельный номер  840 non-null    int32  
 1   mr_id            840 non-null    int64  
 2   tabelnum         840 non-null    float64
dtypes: float64(1), int32(1), int64(1)
memory usage: 23.0 KB


In [40]:
validation2['Табельный номер']=validation2['Табельный номер'].astype(int)

In [41]:
validation2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416072 entries, 0 to 416071
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Дата и время поездки  416072 non-null  object
 1   Терминал              416072 non-null  int64 
 2   Номер билета          416072 non-null  int64 
 3   Номер кристалла       365409 non-null  object
 4   TCAT                  416070 non-null  object
 5   NUM_TRIP              416072 non-null  int64 
 6   Код Маршрута          416072 non-null  int64 
 7   Табельный номер       416072 non-null  int32 
 8   Код предприятия       416072 non-null  int64 
 9   mr_id                 416072 non-null  int64 
 10  time                  416072 non-null  int32 
 11  start                 416072 non-null  int32 
 12  finish                416072 non-null  int32 
dtypes: int32(4), int64(6), object(3)
memory usage: 38.1+ MB


In [42]:
validation4=validation2.merge(table_num_merge_29,on =['mr_id','Табельный номер'],how='left')
validation4=validation4.sort_values(by=['mr_id','Табельный номер','time'])

In [43]:
res_stop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607727 entries, 0 to 607726
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   mr_id       607727 non-null  int64  
 1   rc_orderby  607727 non-null  int64  
 2   ssId        607727 non-null  int64  
 3   tr_id       607727 non-null  int64  
 4   timenav     607727 non-null  object 
 5   len         607727 non-null  int64  
 6   geometry    607727 non-null  object 
 7   ss_lat      607727 non-null  float64
 8   ss_long     607727 non-null  float64
 9   tabelnum    607307 non-null  float64
 10  rl_id       607727 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 51.0+ MB


In [44]:
validation4['tabelnum']=validation4['tabelnum'].astype(int)

In [45]:
validation4.head()

,Дата и время поездки,Терминал,Номер билета,Номер кристалла,TCAT,NUM_TRIP,Код Маршрута,Табельный номер,Код предприятия,mr_id,time,start,finish,tabelnum
80381,2019-10-16 10:35:51,81526452,3,C502D58F,СОЦ,64,10003456789,11,5,1,38151,38151,80104,1503
80624,2019-10-16 10:36:23,81526452,6,NaN,БАНК,0,10003456789,11,5,1,38183,38151,80104,1503
80324,2019-10-16 10:36:57,81526452,7,118F850F,КШ,0,10003456789,11,5,1,38217,38151,80104,1503
80382,2019-10-16 10:37:07,81526452,8,5D09C9AE,СОЦ,81,10003456789,11,5,1,38227,38151,80104,1503
80383,2019-10-16 10:37:12,81526452,9,551B6DF9,СОЦ,69,10003456789,11,5,1,38232,38151,80104,1503


In [46]:
del validation2

Переводим столбец с временем в формат INT и прописываем для каждого табельного номера время начала и конца

In [47]:
res_stop['time']=pd.to_datetime(res_stop['timenav'])
res_stop['hour']=res_stop['time'].dt.hour
res_stop.loc[res_stop.hour ==0, 'hour'] = 24
res_stop.loc[res_stop.hour ==1, 'hour'] = 25
res_stop.loc[res_stop.hour ==2, 'hour'] = 26
res_stop.loc[res_stop.hour ==3, 'hour'] = 27
res_stop['min']=res_stop['time'].dt.minute
res_stop['sec']=res_stop['time'].dt.second
res_stop['time']=(res_stop['hour'].astype(int))*3600+(res_stop['min'].astype(int))*60+(res_stop['sec'].astype(int))
res_stop2=res_stop[['time','ssId','mr_id','rc_orderby','tabelnum','tr_id','rl_id']]

In [48]:
res_stop['hour'].unique()

array([ 6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21,  9,  5, 11, 12,
       22, 23, 24,  4], dtype=int64)

In [49]:
res_stop2['mr_id'].unique()

array([100, 103,  10, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 126, 127, 129,  12, 130, 132, 133, 134, 135, 136, 137,
       138, 139, 140, 141, 142, 143, 144, 145, 146,  14, 151, 152,  15,
       176,  17, 187,  18, 199,   1,  20,  23,  24,  25,  26,  27,  28,
        30,  31,  32,  34,  35,  36,  37,  38,  39,  43,  44,  45,  46,
        47,  48,  49,  59,  60,  61,  62,  64,  68,  70,  71,  73,  81,
        97,  98,  99], dtype=int64)

In [50]:
res_stop2.head()

,time,ssId,mr_id,rc_orderby,tabelnum,tr_id,rl_id
0,21982,1460,100,1,10193.0,500000338,59565
1,21984,1460,100,1,10193.0,500000338,59565
2,21986,1460,100,1,10193.0,500000338,59565
3,21987,1460,100,1,10193.0,500000338,59565
4,21993,1460,100,1,10193.0,500000338,59565


In [51]:
res_stop2['rc_orderby2'] = res_stop2.groupby(['mr_id','tabelnum','tr_id','rl_id'])['rc_orderby'].shift(-1).astype('Int64')

<ipython-input-51-ab181c24d7ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_stop2['rc_orderby2'] = res_stop2.groupby(['mr_id','tabelnum','tr_id','rl_id'])['rc_orderby'].shift(-1).astype('Int64')


In [52]:
res_stop2=res_stop2[~(res_stop2['rc_orderby2']==res_stop2['rc_orderby'])]

In [53]:
res_stop2['time_start']=res_stop2['time']
res_stop2['time_finish'] = res_stop2.groupby(['mr_id','tabelnum','tr_id'])['time'].shift(-1).astype('Int64')

In [54]:
res_stop2.drop(['rc_orderby2'], axis=1)

,time,ssId,mr_id,rc_orderby,tabelnum,tr_id,rl_id,time_start,time_finish
73,22680,1460,100,1,10193.0,500000338,59565,22680,22802
74,22802,284,100,2,10193.0,500000338,59565,22802,22969
75,22969,277,100,4,10193.0,500000338,59565,22969,23113
76,23113,283,100,5,10193.0,500000338,59565,23113,23211
77,23211,278,100,6,10193.0,500000338,59565,23211,23302
...,...,...,...,...,...,...,...,...,...
607721,71558,286,99,35,1301.0,500000420,61766,71558,71617
607722,71617,287,99,36,1301.0,500000420,61766,71617,71717
607723,71717,263,99,37,1301.0,500000420,61766,71717,72068
607724,72068,264,99,38,1301.0,500000420,61766,72068,72158


In [55]:
validation4['tr_id']='';

In [56]:
validation4=validation4[validation4['mr_id'].isin(res_stop2['mr_id'].unique())]

Смотрим сколько маршрутов у нас есть и в валидациях, и в треках

In [57]:
len(validation4['mr_id'].unique())

80

In [58]:
validation4[validation4['mr_id']==1]['tabelnum'].unique()

array([1503, 1604, 1228,  204,  735, 1408,  895, 1591, 1674, 1456, 1202,
       1168, 1515, 1329, 1357, 1277,  404, 1686, 1287,  725])

In [59]:
res_stop2[res_stop2['mr_id']==1]['tabelnum'].unique()

array([1503., 1228., 1329.,  404., 1686.,  725., 1591., 1168., 1202.,
        735.,  895., 1456., 1287., 1408., 1357., 1277., 1674.,  204.,
       1515., 1604.])

In [60]:
validation4['tabelnum'].unique()

array([ 1503,  1604,  1228,   204,   735,  1408,   895,  1591,  1674,
        1456,  1202,  1168,  1515,  1329,  1357,  1277,   404,  1686,
        1287,   725,  2618,  1209,   300,  1139,   380,   106,  3778,
        1821,   488,  1785,  2630,  1752,   680,  1363,  4141,  3611,
        4156,   637,   840,   531,  1813,  3909,   639,  2813,  3892,
        2491,   670,   473,  3753,  4040,  2640,  3715,  1677,  2408,
         664,  1210,  1657,  1412,   804,  1161,   405,  1259,  1337,
        1175,  1673,  1351,  1350,  1608,  1235,  1203,  1196,  1368,
        1385,  1440,  1374,  1399,  1323,  1037,   402,  1671,   594,
        1059,  1463,    70,   341,  1014,  1244,  1518,  1397,  1386,
        1688,  1703,  1187,  1243,  1178,  1364,  1174,  1371,   128,
         868,   550,   630,  1092,  1163,   950,  1476,  1062,   319,
         565,   105,  1528,   120,  1239,   122,  1019,  1190, 10058,
       10226, 10195, 10176, 10262, 10219, 10253, 10049,   410,   976,
         938,  1061,

In [61]:
k=0
res=validation4
result = []
result2 = []
stops=pd.DataFrame()
for index, row in validation4.iterrows():
    stops=res_stop2[(res_stop2['mr_id']==row['mr_id'])
                    &(res_stop2['tabelnum']==row['tabelnum'])
                    &(res_stop2['time_start']<=row['time'])
                    &((res_stop2['time_finish']>row['time'])|(res_stop2['time_finish'].isna()))]
    if stops.empty:
        result.append(0)
        result2.append(0)
    else:
        stop_unic=stops['ssId'].values
        result.append(stop_unic[0])
        rl_id=stops['rl_id'].values
        result2.append(rl_id[0])
res['ssId']=result
res['rl_id']=result2
FOR_MATRIX=res

In [62]:
len(res[res['ssId']==0])

49901

In [63]:
stops=pd.read_csv(path+'stops.csv', sep = ',', encoding='utf-8')
stops.head()

,ssId,ss_name,ss_lat,ss_long,geometry
0,2,Завод Электрощит,53.381994,50.166450,0101000020E6100000A835CD3B4E15494055A2EC2DE5B0...
1,3,пос.Южный,53.372294,50.177035,0101000020E6100000A99F3715A91649409E3F6D54A7AF...
2,4,Развилка,53.370268,50.188537,0101000020E6100000008BFCFA21184940AF601BF164AF...
3,5,Дома ЭМО,53.365598,50.197635,0101000020E6100000C1C58A1A4C194940D3BD4EEACBAE...
4,6,31 км,53.355974,50.209239,0101000020E61000001C06F357C81A4940F81C588E90AD...


In [64]:
res['ssId']=res['ssId'].astype(int)
res2=res.merge(stops, on='ssId', how='left')
res2['c']=1
res2[res2['ssId']==0]['mr_id'].unique()
rl_orderby=res_stop[['ssId','rl_id','mr_id','rc_orderby']].drop_duplicates()
res2=res2.merge(rl_orderby, on=['ssId','rl_id','mr_id'], how='left')
len(res2[res2['ssId']==0])

49901

## ИЗМЕНИИТЬ НАЗВАНИЯ ФАЙЛА!!! НЕ ЗАБЫТЬ

In [68]:
res2.to_csv(path+'For_matrix_2019_10_16.csv')

In [66]:
13756/59371

0.23169560896734095

In [67]:
marsh=pd.read_csv(path+'Маршруты.csv', sep = ';', encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '.\\done_oct\\Маршруты.csv'

In [ ]:
marsh.head()

In [ ]:
res2=res2.merge(marsh, on='Код Маршрута', how='left')

In [ ]:
res2.to_csv('stops_2020_08_03.csv')

In [ ]:
for_karta=res2.groupby(['Код Маршрута','Табельный номер','ssId','ss_lat','ss_long','start','finish']).agg({'c': np.sum}).reset_index()

In [ ]:
res2.head()

In [ ]:
gdf = gpd.GeoDataFrame(
    for_karta, geometry=gpd.points_from_xy(x=for_karta.ss_long, y=for_karta.ss_lat)
)
gdf.crs='epsg:4326'

In [ ]:
gdf=gdf.rename(columns={'Код Маршрута':'code_marsh','Табельный номер':'tabelnum'})

In [ ]:
gdf.groupby(['code_marsh','tabelnum']).apply(lambda x: x.to_file(fr"./res/{x.name}.json", index=False, driver="GeoJSON"))

In [ ]:
for_karta2=res2.groupby(['Код Маршрута','ssId','ss_lat','ss_long','start','finish']).agg({'c': np.sum}).reset_index()
gdf = gpd.GeoDataFrame(
    for_karta2, geometry=gpd.points_from_xy(x=for_karta2.ss_long, y=for_karta2.ss_lat)
)
gdf=gdf.rename(columns={'Код Маршрута':'code_marsh'})
gdf.crs='epsg:4326'
gdf.groupby(['code_marsh','start','finish']).apply(lambda x: x.to_file(fr"./res/{x.name}.json", index=False, driver="GeoJSON"))

In [ ]:
for_karta3=res2.groupby(['ssId','ss_name','ss_lat','ss_long']).agg({'c': np.sum}).reset_index()
for_karta3.to_csv('./res/stops_2020_08_03.csv')

In [ ]:
for_karta3=res2.groupby(['ssId','ss_name','ss_lat','ss_long']).agg({'c': np.sum}).reset_index()
for_karta3=for_karta3.groupby(['ssId','ss_name','ss_lat','ss_long']).agg({'c': np.sum}).reset_index()
gdf = gpd.GeoDataFrame(
    for_karta3, geometry=gpd.points_from_xy(x=for_karta3.ss_long, y=for_karta3.ss_lat)
)
gdf.crs='epsg:4326'
gdf.to_file(fr"./res/stops_2020_08_03.json", index=False, driver="GeoJSON")